In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_xml('https://api.worldbank.org/v2/country/all/indicator/SP.URB.TOTL.IN.ZS?date=2019&format=xml&per_page=300')
df

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,Urban population (% of total population),Africa Eastern and Southern,AFE,2019,36.291322,NaN,NaN,0
1,Urban population (% of total population),Africa Western and Central,AFW,2019,47.242853,NaN,NaN,0
2,Urban population (% of total population),Arab World,ARB,2019,59.201646,NaN,NaN,0
3,Urban population (% of total population),Caribbean small states,CSS,2019,51.369979,NaN,NaN,0
4,Urban population (% of total population),Central Europe and the Baltics,CEB,2019,62.438139,NaN,NaN,0
...,...,...,...,...,...,...,...,...
261,Urban population (% of total population),Virgin Islands (U.S.),VIR,2019,95.832000,NaN,NaN,0
262,Urban population (% of total population),West Bank and Gaza,PSE,2019,76.440000,NaN,NaN,0
263,Urban population (% of total population),"Yemen, Rep.",YEM,2019,37.273000,NaN,NaN,0
264,Urban population (% of total population),Zambia,ZMB,2019,44.072000,NaN,NaN,0
